In [21]:
import airfoils as af
import matplotlib.pyplot as plt
import numpy as np
import potential as pot
from wing import Wing

# Airfoil

In [40]:
airfoil = af.Airfoil.NACA4("4415", n_points=200)
af_xs, af_ys = airfoil.all_points
plt.scatter(af_xs, af_ys, 1)
plt.axis('scaled')

alpha0 = -4 * np.pi / 180

# Wing Geometry

In [41]:
def linSpan(sp, Ni):
    return np.linspace(0, sp, Ni)

def uniformChord(Ni, ch=1):
    return ch * np.ones(Ni)

In [53]:
N = 50
M = 3
%matplotlib qt
wg = Wing(
    airfoil=airfoil,
    isSymmetric=True,
    span=6,
    alpha=2,
    beta=0,
    Ni=N,
    Nj=M,
    sweepOffset=0,
    dihAngle=0,
    chordFun=uniformChord,
    spanFun=linSpan,
    ground_clearence = 100
)

## Wind

In [54]:
Umag = 16
dens = 1.2

# Ground Effect

In [55]:
def ground_effect(x, y, z, i, j, panel):

    U1, U1st = pot.hshoeSL2(x, y, z, i, j, panel)
    U2 , U2st = pot.hshoeSL2(x, y, -z, i, j, panel)

    U_ind = np.array([U1[0] + U2[0],
                      U1[1] + U2[1],
                      U1[2] - U2[2]])
    U_ind_st = np.array([U1st[0] + U2st[0],
                         U1st[1] + U2st[1],
                         U1st[2] - U2st[2]])
    return U_ind , U_ind_st

In [56]:
L_ge, D_ge, Gammas, w_ind = wg.solveWing(alpha0, Umag, dens, ground_effect)
print(f"For the whole Wing\nL:{L_ge}\t|\tD:{2*D_ge}")
print(f"CL:{L_ge/(dens*(Umag**2)*wg.wingArea)}\t|\tCD_ind:{D_ge/(dens*(Umag**2)*wg.wingArea)}")

For the whole Wing
L:443.074959209243	|	D:23.0295116449018
CL:0.24038354991820912	|	CD_ind:0.006247154851590116


In [50]:
LvsGdist = []
DvsGdist = []
GammavsGdist = []
WindvsGdist = []
groundDist = np.linspace(0.2, 20,20)
for gDist in groundDist:
    wg.set_groundClearence(gDist)
    L , D , Gamma , w = wg.solveWing(alpha0,Umag,dens,ground_effect)
    LvsGdist.append(L)
    DvsGdist.append(D)
    GammavsGdist.append(Gamma)
    WindvsGdist.append(w)

In [52]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 6))
fig.suptitle('Ground Distance effect on Lift And Drag')

ax1.plot(groundDist, LvsGdist)
ax1.set_xlabel("Ground Distance")
ax1.set_title("Lift")
ax1.minorticks_on()
ax1.grid()

ax2.plot(groundDist, DvsGdist)
ax2.set_xlabel("Ground Distance")
ax2.set_title("Drag")
ax2.minorticks_on()
ax2.grid()

ax3.plot(groundDist, np.divide(LvsGdist, DvsGdist))
ax3.set_xlabel("Ground Distance")
ax3.set_title("L/D")
ax3.minorticks_on()
ax3.grid()

fig.show()